In [2]:
from Tkinter import *
import random
import copy

def snakeInit(game, boardSize):
    class Struct(): pass
    game.data = Struct()
    game.data.board = snakeInitialBoard(game, boardSize)
    game.data.width = game.data.height = 30*boardSize
    game.data.DIRS = [(1,0),(-1,0),(0,1),(0,-1)]
    game.data.direction = random.choice(game.data.DIRS)
    game.data.snakeLength = 1
    game.data.gameOver = False
    game.data.boardSize = boardSize
    updateSnakeBoard(game)
    snakeRedrawAll(game)
    snakeTimerFired(game)
    
def snakeRun(boardSize = 10):
    printSnakeRules()
    root = Tk()
    game = Canvas(width = 30*boardSize + 10, height = 30*boardSize + 10) #n*n cells with a 10 pixel buffer
    game.pack()
    root.bind('<Key>', lambda event: snakeKeyPressed(event, game))
    snakeInit(game, boardSize)
    
def printSnakeRules():
    print '''
    Snake!
    Use the arrow keys to move the snake.
    Eat food to grow.
    Stay on the board!
    And don't crash into yourself!
    
    Hit q to restart.
    '''
    
def snakeInitialBoard(game, boardSize):
    board = []
    for i in xrange(boardSize): # for a n x n board
        board.append([0 for j in xrange(boardSize)])
    startingLocation = (random.randrange(boardSize), random.randrange(boardSize))
    game.data.headLocation = startingLocation
    board[startingLocation[0]][startingLocation[1]] = 1 #random Head of the snake
    foodLocation = (random.randrange(boardSize), random.randrange(boardSize))
    while foodLocation == startingLocation:
        foodLocation = (random.randrange(boardSize), random.randrange(boardSize))
    board[foodLocation[0]][foodLocation[1]]=-1
    return board
    
def updateSnakeBoard(game):
    initialBoard = game.data.board
    (drow,dcol) = game.data.direction
    (headY,headX) = game.data.headLocation
    if inRange(game.data.board ,game.data.headLocation, game.data.direction):
        if game.data.board[headY + drow][headX + dcol] == 0:
            moveSnake(game)
        elif game.data.board[headY + drow][headX + dcol] == -1:
            eatSnake(game)
        else:
            game.data.gameOver = True
    else: 
        game.data.gameOver = True
        
def inRange(board, location, move):
    width = len(board)
    height = len(board[0])
    return ((0<= location[0] + move[0] < height) and (0<= location[1] + move[1] < width))
        
def moveSnake(game):# move only head and decrement everything else
    drow, dcol = game.data.direction
    game.data.board[game.data.headLocation[0]+drow][game.data.headLocation[1]+dcol] = \
    game.data.board[game.data.headLocation[0]][game.data.headLocation[1]] #moving the head acordingly
    game.data.headLocation = (game.data.headLocation[0]+drow,game.data.headLocation[1]+dcol)
    for row in xrange(len(game.data.board)):
        for col in xrange(len(game.data.board[0])):
            if game.data.board[row][col] > 0 and (row,col) != game.data.headLocation:
                game.data.board[row][col] -= 1
                
def eatSnake(game):#just incoorporate the next tile into the snake
    game.data.snakeLength += 1
    drow, dcol = game.data.direction
    game.data.headLocation = (game.data.headLocation[0]+drow,game.data.headLocation[1]+dcol)
    game.data.board[game.data.headLocation[0]][game.data.headLocation[1]] = game.data.snakeLength
    newFood = (random.randrange(game.data.boardSize), random.randrange(game.data.boardSize))
    while game.data.board[newFood[0]][newFood[1]] != 0:
        newFood = (random.randrange(game.data.boardSize), random.randrange(game.data.boardSize))
    game.data.board[newFood[0]][newFood[1]] = -1
    
def snakeRedrawAll(game):
    game.delete(ALL)
    snakeDraw(game)

def snakeDraw(game):
    for row in xrange(len(game.data.board)):
        for col in xrange(len(game.data.board[0])):
            if game.data.board[row][col] == 0:
                drawEmptyCell(game, row, col )
            elif game.data.board[row][col] == -1:
                drawFoodCell(game, row, col)
            else: drawSnakeCell(game, row, col)
            
def drawEmptyCell(game, row, col):
    cellSize = int(game.data.width / float(game.data.boardSize))
    game.create_rectangle(col*cellSize + 5, row*cellSize + 5, (col+1)*cellSize + 5, (row+1)*cellSize + 5, fill = 'White')
    
def drawFoodCell(game, row, col):
    cellSize = int(game.data.width / float(game.data.boardSize))
    game.create_rectangle(col*cellSize + 5, row*cellSize + 5, (col+1)*cellSize + 5, (row+1)*cellSize + 5, fill = 'White')
    game.create_oval(col*cellSize + 5, row*cellSize + 5, (col+1)*cellSize + 5, (row+1)*cellSize + 5, fill = 'bisque')
    
def drawSnakeCell(game, row, col):
    cellSize = int(game.data.width / float(game.data.boardSize))
    game.create_rectangle(col*cellSize + 5, row*cellSize + 5, (col+1)*cellSize + 5, (row+1)*cellSize + 5, fill = 'White')
    game.create_oval(col*cellSize + 5, row*cellSize + 5, (col+1)*cellSize + 5, (row+1)*cellSize + 5, fill = 'royal blue')
    
def snakeTimerFired(game):
    game.data.movesInput = 0
    delay = 200 #ms
    if not game.data.gameOver:
        updateSnakeBoard(game)
        snakeRedrawAll(game)
        game.after(delay, lambda : snakeTimerFired(game))
    else:
        print 'Game Over!'
        print 'Hit q to restart.'

    
def snakeKeyPressed(event, game):
    movesInput = game.data.movesInput
    # only new directions perpendicular to the current path are allowed
    if event.keysym == 'Up' and (game.data.direction[0] == 0) and movesInput == 0:
        game.data.direction = (-1, 0)
        movesInput += 1
    elif event.keysym == 'Down' and (game.data.direction[0] == 0) and movesInput == 0:
        game.data.direction = (1, 0)
        movesInput += 1
    elif event.keysym == 'Right' and (game.data.direction[1] == 0) and movesInput == 0:
        game.data.direction = (0,1)
        movesInput += 1
    elif event.keysym == 'Left' and (game.data.direction[1] == 0) and movesInput == 0:
        game.data.direction = (0,-1)
        movesInput += 1
    elif event.keysym == 'q':
        game.delete(ALL)
        snakeInit(game, game.data.boardSize)
    
    
# Tetris

def tetrisRun(rows = 15, cols = 10):
    root = Tk()
    game = Canvas(width = cols*30 + 60, height = rows*30 + 60)
    game.pack()
    root.resizable(width=0, height=0)
    root.bind('<Key>', lambda event: tetrisKeyPressed(event, game))
    class Struct():pass
    game.data = Struct()
    game.data.rows = rows
    game.data.cols = cols
    game.data.cellWidth = 30
    game.data.width = cols*game.data.cellWidth + game.data.cellWidth*2
    game.data.height = rows*game.data.cellWidth + game.data.cellWidth*2
    root.bind('<Key>', lambda event: tetrisKeyPressed(event, game))
    tetrisInit(game)
    tetrisTimerFired(game)
    
def tetrisInit(game):
    game.data.emptyColor = 'blue'
    game.data.board = emptyBoard(game.data.rows, game.data.cols, game.data.emptyColor)
    board = game.data.board
    rows, cols = game.data.rows, game.data.cols
    #Seven "standard" pieces (tetrominoes)
    iPiece = [
        [ True,  True,  True,  True]
      ]
  
    jPiece = [
        [ True, False, False ],
        [ True, True,  True]
      ]
  
    lPiece = [
        [ False, False, True],
        [ True,  True,  True]
      ]
  
    oPiece = [
        [ True, True],
        [ True, True]
      ]
  
    sPiece = [
        [ False, True, True],
        [ True,  True, False ]
      ]
  
    tPiece = [
        [ False, True, False ],
        [ True,  True, True]
      ]

    zPiece = [
        [ True,  True, False ],
        [ False, True, True]
      ]
    game.data.pieces = [ iPiece, jPiece, lPiece, oPiece, sPiece, tPiece, zPiece ]
    game.data.pieceColors = [ "red", "yellow", "magenta", "pink", "cyan", "green", "orange" ]
    # pre-load a few cells with known colors for testing purposes
    #board[0][0] = "red" # top-left is red
    #board[0][cols-1] = "white" # top-right is white
    #board[rows-1][0] = "green" # bottom-left is green
    #board[rows-1][cols-1] = "gray" # bottom-right is gray
    game.data.gameOver = False
    game.data.score = 0
    game.data.pause = False
    tetrisNewFallingPiece(game)
    tetrisRedrawAll(game)
    
def emptyBoard(rows, cols, emptyValue):
    board = []
    for i in xrange(rows):
        board.append([emptyValue for j in xrange(cols)])
    return board
    
def tetrisRedrawAll(game):
    game.delete(ALL)
    tetrisDrawGame(game)
    tetrisDrawScore(game)
    if game.data.gameOver:
        tetrisDrawGameOver(game)
    
def tetrisDrawGame(game):
    game.create_rectangle(0,0,game.data.width,game.data.height, fill = 'orange')
    tetrisDrawBoard(game)
    drawFallingPiece(game)
    
def tetrisDrawBoard(game):
    for i in xrange(len(game.data.board)):
        for j in xrange(len(game.data.board[0])):
            tetrisDrawCell(game, j*game.data.cellWidth + game.data.cellWidth, 
                           i*game.data.cellWidth + game.data.cellWidth, game.data.cellWidth, 
                           game.data.board[i][j])
            
def tetrisDrawCell(canvas, top, left, size, fill):
    canvas.create_rectangle(top , left, top + size, left + size, fill = 'black' )
    canvas.create_rectangle(top + 2, left + 2, top + size - 2, left + size - 2, fill = fill ) #for a 5 pixel outline of each cell
    
def drawFallingPiece(game):
    row = game.data.fallingPieceRow
    col = game.data.fallingPieceCol
    cellWidth = game.data.cellWidth
    for i in xrange(len(game.data.fallingPiece)):
        for j in xrange(len(game.data.fallingPiece[0])):
            if game.data.fallingPiece[i][j] == True:
                tetrisDrawCell(game, (col + j)*cellWidth + cellWidth, (row + i)*cellWidth + cellWidth, cellWidth, game.data.fallingPieceColor)    
    
def tetrisNewFallingPiece(game):
    newFallingPiece = random.choice(game.data.pieces)
    newFallingPieceColor = random.choice(game.data.pieceColors)
    game.data.fallingPiece = newFallingPiece 
    game.data.fallingPieceColor = newFallingPieceColor
    fallingPieceWidth = len(newFallingPiece[0])
    game.data.fallingPieceRow = 0
    game.data.fallingPieceCol = game.data.cols/2 - fallingPieceWidth/2
    
def tetrisKeyPressed(event, game):
    drow,dcol = 0,0
    if game.data.pause == False:
        if event.keysym == 'Left':
            drow,dcol = 0,-1
        elif event.keysym == 'Right':
            drow,dcol = 0,1
        elif event.keysym == 'Down':
            drow,dcol = 1,0
        elif event.keysym == 'Up':
            rotateFallingPiece(game)
    if event.keysym == 'r':
        tetrisInit(game)
    elif event.keysym == 'p':
        game.data.pause = not game.data.pause
    moveFallingPiece(game,drow,dcol)
    tetrisRedrawAll(game)
    
def moveFallingPiece(game, drow, dcol):
    game.data.fallingPieceCol += dcol
    game.data.fallingPieceRow += drow
    if not fallingPieceIsLegal(game):
        game.data.fallingPieceCol -= dcol
        game.data.fallingPieceRow -= drow
        return False
    return True
        
def fallingPieceIsLegal(game):
    maxRows = game.data.rows
    maxCols = game.data.cols
    initialRow = game.data.fallingPieceRow
    initialCol = game.data.fallingPieceCol
    for i in xrange(len(game.data.fallingPiece)):
        for j in xrange(len(game.data.fallingPiece[0])):
            if not (0<=(initialRow + i)<maxRows):
                return False
            elif not (0<= (initialCol + j)<maxCols):
                return False
            elif game.data.fallingPiece[i][j] == True and \
             game.data.board[initialRow + i][initialCol + j] != game.data.emptyColor:
                return False
    return True
    
def rotateFallingPiece(game):
    oldPiece = game.data.fallingPiece
    oldRow, oldCol = game.data.fallingPieceRow, game.data.fallingPieceCol
    newPiece = []
    for i in xrange (len(oldPiece[0])-1,-1,-1):
        newPiece.append([oldPiece[j][i] for j in xrange(len(oldPiece))])
    drow = len(newPiece) - len(oldPiece)
    dcol = len(newPiece[0]) - len(oldPiece[0])
    newRow = oldRow - (drow/abs(drow))*(abs(drow)/2) #fix the rounding down so that
    newCol = oldCol - (dcol/abs(dcol))*(abs(dcol)/2) #it rounds towards zero rather than -1 
    game.data.fallingPieceRow, game.data.fallingPieceCol = newRow, newCol
    game.data.fallingPiece = newPiece
    if not fallingPieceIsLegal(game):
        game.data.fallingPieceRow, game.data.fallingPieceCol = oldRow, oldCol
        game.data.fallingPiece = oldPiece
    
def tetrisTimerFired(game):
    if game.data.pause == False:
        if not moveFallingPiece(game, 1, 0) and game.data.gameOver == False :
            placeFallingPiece(game)
            tetrisNewFallingPiece(game)
            if not fallingPieceIsLegal(game):
                game.data.gameOver = True
                tetrisDrawGameOver(game)
    delay = 500
    tetrisRedrawAll(game)
    game.after(delay, lambda : tetrisTimerFired(game))
    
def placeFallingPiece(game):
    for i in xrange(len(game.data.fallingPiece)):
        for j in xrange(len(game.data.fallingPiece[0])):
            if game.data.fallingPiece[i][j]==True:
                game.data.board[i + game.data.fallingPieceRow][j + game.data.fallingPieceCol] = \
                game.data.fallingPieceColor
                
def tetrisDrawGameOver(game):
    game.create_text(game.data.width/2, game.data.height/2, text = 'Game Over', font = 'Helvetica 30 bold', fill = 'white')
    
def removeFullRows(game):
    fullRows = 0
    newBoard = []
    for oldRow in xrange(len(game.data.board)-1, -1, -1):
        if len(set(game.data.board[oldRow]))>1:
            newBoard = [game.data.board[oldRow]] + newBoard
        else: 
            fullRows += 1
    for emptyRow in xrange(fullRows):
        newBoard = [[game.data.emptyColor for col in game.data.cols]] + newBoard
    game.data.board = newBoard
    game.data.score += fullRows**2
    
def tetrisDrawScore(game):
    game.create_text(5,5, anchor = 'nw', text ='Score \n' + str(game.data.score), fill = 'Black', font = 'Helvetica 15 bold underline')
    
# Shape Inversion

def shapeRun():
    root = Tk()
    game = Canvas(width = 500, height = 700)
    game.pack()
    root.resizable(width=0, height=0)
    root.bind('<Button-1>', lambda event: shapeButtonPressed(event, game))
    root.bind('<Key>', lambda event: shapeKeyPressed(event, game))
    class Struct(): pass
    game.data = Struct()
    game.data.width = 500
    game.data.height = 700
    game.data.shapeDiameter = 50
    game.data.shapes = ['triangle', 'square', 'pentagon', 'hexagon', 'circle']
    shapeInit(game)
    shapeTimerFired(game)
    
def shapeInit(game):
    game.data.shapes = randomShapeLineup(game)
    game.data.fallingShapes = randomShapeLineup(game)
    game.data.score = 0
    game.data.fallingPercentage = 0
    game.data.percentageStep = .007
    game.data.firstIndex = -1
    game.data.secondIndex = -1
    game.data.win = False
    game.data.lose = False

    
def randomShapeLineup(game):
    return list(sorted(game.data.shapes, key = lambda elem: random.randint(0,10)))
    
def shapeRedrawAll(game):
    game.delete(ALL)
    drawShapes(game, game.data.fallingShapes, game.data.fallingPercentage*game.data.height)
    drawShapes(game, game.data.shapes, game.data.height)
    drawSelections(game)
    drawScore(game)
    if game.data.win: shapesWin(game)
    if game.data.lose: shapesLose(game)
    
def shapeTimerFired(game):
    game.data.percentageStep = (0.007)*(1.2)**game.data.score
    if game.data.score == 5:
        game.data.win = True
    elif game.data.score >= 0:
        shapeRedrawAll(game)
        if game.data.fallingPercentage < 1 and (game.data.fallingShapes != game.data.shapes):
            game.data.fallingPercentage += game.data.percentageStep
        elif game.data.fallingShapes == game.data.shapes:
            game.data.score+=1
            game.data.fallingShapes = randomShapeLineup(game)
            game.data.fallingPercentage = 0
        else: 
            game.data.fallingPercentage = 0
            game.data.score -= 1
            game.data.fallingShapes = randomShapeLineup(game)
    else: game.data.lose = True
    shapeRedrawAll(game)
    delay = 30
    game.after(delay, lambda : shapeTimerFired(game))
    
def drawShapes(game, shapes, height): #height defines the bottom
    cellWidth = game.data.width/len(shapes)
    cellHeight = game.data.shapeDiameter
    for i in xrange(len(shapes)):
        centerX = (i*cellWidth + (i+1)*cellWidth)/2
        centerY = (height + height-cellHeight)/2
        if shapes[i] == 'triangle':
            drawTriangle(game, height, i)
        elif shapes[i] == 'square':
            drawSquare(game, height, i)
        elif shapes[i] == 'pentagon':
            drawPentagon(game, height, i)
        elif shapes[i] == 'hexagon':
            drawHexagon(game, height, i)
        elif shapes[i] == 'circle':
            drawCircle(game, height, i)
            
def drawTriangle(game, height, index):
    i = index
    cellWidth = game.data.width/len(game.data.shapes)
    cellHeight = game.data.shapeDiameter
    centerX = (i*cellWidth + (i+1)*cellWidth)/2
    centerY = (height + height-cellHeight)/2
    game.create_polygon(centerX - cellHeight/2, height,
                        centerX + cellHeight/2, height, 
                        centerX, height - cellHeight, fill = 'green', outline = 'green')
    
def drawSquare(game, height, index):
    i = index
    cellWidth = game.data.width/len(game.data.shapes)
    cellHeight = game.data.shapeDiameter
    centerX = (i*cellWidth + (i+1)*cellWidth)/2
    centerY = (height + height-cellHeight)/2
    game.create_rectangle(centerX - cellHeight/2, centerY - cellHeight/2,
                          centerX + cellHeight/2, centerY + cellHeight/2, fill = 'pink', outline = 'pink')
    
def drawPentagon(game, height, index):
    i = index
    cellWidth = game.data.width/len(game.data.shapes)
    cellHeight = game.data.shapeDiameter
    centerX = (i*cellWidth + (i+1)*cellWidth)/2
    centerY = (height + height-cellHeight)/2
    game.create_rectangle(centerX - cellHeight/2, centerY + cellHeight/2, 
                          centerX + cellHeight/2, height - 2*cellHeight/3, 
                          fill = 'orange', outline = 'orange')
    game.create_polygon(centerX - cellHeight/2, height - 2*cellHeight/3,
                        centerX, height - cellHeight,
                        centerX + cellHeight/2, height - 2*cellHeight/3,  
                        fill = 'orange')

def drawHexagon(game, height, index):
    i = index
    cellWidth = game.data.width/len(game.data.shapes)
    cellHeight = game.data.shapeDiameter
    centerX = (i*cellWidth + (i+1)*cellWidth)/2
    centerY = (height + height-cellHeight)/2
    game.create_rectangle((i+.25)*cellWidth, height,
                          (i+.75)*cellWidth, height - cellHeight,
                          fill = 'red', outline = 'red')
    game.create_polygon((i+.25)*cellWidth, height - cellHeight, 
                         i*cellWidth, height - cellHeight/2, 
                         (i+.25)*cellWidth, height, 
                         fill ='red')
    game.create_polygon(((i+.75)*cellWidth, height - cellHeight), 
                         (i+1)*cellWidth, height - cellHeight/2, 
                         (i+.75)*cellWidth, height, 
                         fill = 'red')

def drawCircle(game, height, index):
    i = index
    cellWidth = game.data.width/len(game.data.shapes)
    cellHeight = game.data.shapeDiameter
    centerX = (i*cellWidth + (i+1)*cellWidth)/2
    centerY = (height + height-cellHeight)/2
    game.create_oval(centerX - cellHeight/2, centerY - cellHeight/2,
                     centerX + cellHeight/2, centerY + cellHeight/2, 
                     fill = 'blue', outline = 'blue')
     
def shapeButtonPressed(event, game):
    if game.data.firstIndex == -1:
        game.data.firstIndex = event.x/(game.data.width/len(game.data.shapes))
    elif game.data.secondIndex == -1:
        game.data.secondIndex = event.x/(game.data.width/len(game.data.shapes))
        if game.data.secondIndex >= len(game.data.shapes):
            game.data.secondIndex = -1
        elif game.data.secondIndex != game.data.firstIndex:
            game.data.shapes[game.data.firstIndex], game.data.shapes[game.data.secondIndex] = \
            game.data.shapes[game.data.secondIndex], game.data.shapes[game.data.firstIndex]
            game.data.firstIndex = -1
            game.data.secondIndex = -1
        else: game.data.secondIndex ,game.data.firstIndex = -1,-1
        
def shapeKeyPressed(event, game):
    if event.keysym == 'r':
        shapeInit(game)
        
def drawSelections(game):
    cellWidth = game.data.width/len(game.data.shapes)
    cellHeight = game.data.shapeDiameter
    if game.data.firstIndex > -1:
        centerX = (game.data.firstIndex*cellWidth + (game.data.firstIndex + 1)*cellWidth)/2
        centerY = (game.data.height + game.data.height - cellHeight)/2
        radius = 5
        game.create_oval(centerX - radius, centerY - radius, centerX + radius, centerY + radius, fill = 'black')
        
def drawScore(game):
    game.create_text(game.data.width/2, 0 , text ='Score \n' + str(game.data.score), fill = 'Black', font = 'Helvetica 15 bold underline')    
            
def shapesWin(game):
    game.create_text(game.data.width/2, game.data.height/2, text = 'You Win!', font = 'Helvetica 30 bold', fill = 'Cyan')
    
def shapesLose(game):
    game.create_text(game.data.width/2, game.data.height/2, text = 'You Lose!', font = 'Helvetica 35 bold', fill = 'Cyan')

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Game Over!')? (<ipython-input-2-adf71683c894>, line 125)